# Restaurant Performance & Menu Optimization Analysis

# Project Overview

This project analyzes a quarter’s worth of transactional order data from a fictitious restaurant serving international cuisine to evaluate menu performance, revenue drivers, and operational efficiency. The goal is to move beyond descriptive insights and deliver clear, data-backed recommendations that improve profitability and operational decision-making.

Using transaction-level order data, this analysis identifies underperforming menu items, high-value orders, peak and off-peak demand periods, and cuisine-level opportunities for menu optimization and staffing alignment.

# Business Problem

Restaurants often face margin pressure due to menu bloat, inefficient staffing, and misalignment between customer demand and operational resources. Without data-driven insights, decisions around menu changes and staffing levels are typically based on intuition rather than evidence.

This project addresses the following core questions:

Which menu items and categories truly drive value?

Where are resources being allocated inefficiently?

How can menu and staffing decisions be optimized to improve profitability?

# Project Objectives

1. Identify high- and low-performing menu items based on order volume and revenue contribution.

2. Analyze category- and cuisine-level performance to uncover demand patterns.

3. Examine high-spend orders to understand purchasing behavior and revenue concentration.

4. Evaluate temporal ordering trends to identify peak and off-peak periods.

5. Assess operational efficiency by aligning order volume with staffing demand.

6. Provide actionable recommendations for menu optimization and staffing adjustments.

In [19]:
#Import necessary modules
import pandas as pd
import os
import sqlite3

In [20]:
# Connect to the SQLite database
db_path = os.path.join(os.getcwd(), 'restaurant_performance.db')
conn = sqlite3.connect(db_path)
cursor = conn.cursor()


In [21]:
# Create tables and load data from CSV files
# Create menu_items table
cursor.execute("""
    CREATE TABLE IF NOT EXISTS menu_items (
        menu_item_id INTEGER PRIMARY KEY,
        item_name TEXT,
        category TEXT,
        price REAL
    )
""")

# Create order_details table
cursor.execute("""
    CREATE TABLE IF NOT EXISTS order_details (
        order_details_id INTEGER PRIMARY KEY,
        order_id INTEGER,
        order_date DATE,
        order_time TIME,
        item_id INTEGER
    )
""")

# Load menu_items from CSV
menu_df = pd.read_csv('Restaurant Orders/menu_items.csv')
menu_df.to_sql('menu_items', conn, if_exists='replace', index=False)

# Load order_details from CSV
order_details_df = pd.read_csv('Restaurant Orders/order_details.csv')
order_details_df.to_sql('order_details', conn, if_exists='replace', index=False)

conn.commit()
print("✓ Tables created and data loaded successfully!")

✓ Tables created and data loaded successfully!


In [22]:
# Verify data loaded correctly
cursor.execute("SELECT COUNT(*) FROM order_details")
order_count = cursor.fetchone()[0]

cursor.execute("SELECT COUNT(*) FROM menu_items")
menu_count = cursor.fetchone()[0]

cursor.execute("SELECT MIN(order_date), MAX(order_date) FROM order_details")
date_range = cursor.fetchone()

print(f"Order Details Records: {order_count:,}")
print(f"Menu Items Records: {menu_count}")
print(f"Date Range: {date_range[0]} to {date_range[1]}")

Order Details Records: 12,234
Menu Items Records: 32
Date Range: 1/1/23 to 3/9/23


In [23]:
# Menu Items performance overview
menu_df = pd.read_sql_query("SELECT * FROM menu_items", conn)

print("Menu Items Summary:")
print(f"Total unique menu items: {len(menu_df)}")
print(f"\nMenu Items by Category:")
print(menu_df['category'].value_counts())
print(f"\nPrice Range: ${menu_df['price'].min()} - ${menu_df['price'].max()}")
print(f"\nMenu items with missing data:")
print(menu_df[menu_df.isnull().any(axis=1)])

Menu Items Summary:
Total unique menu items: 32

Menu Items by Category:
category
Mexican     9
Italian     9
Asian       8
American    6
Name: count, dtype: int64

Price Range: $5.0 - $19.95

Menu items with missing data:
Empty DataFrame
Columns: [menu_item_id, item_name, category, price]
Index: []


- The menu is evenly distributed across four cuisines, with Mexican and Italian offering the largest item variety.
- Menu prices range from $5.00 to $19.95, allowing for both high- and low-ticket order behavior.

In [24]:
# Check for missing values and data quality issues
print("=" * 60)
print("MISSING VALUES ANALYSIS")
print("=" * 60)

# Missing values in order_details
cursor.execute("""
    SELECT 
        COUNT(*) as total_records,
        COUNT(CASE WHEN item_id IS NULL THEN 1 END) as null_item_ids,
        COUNT(CASE WHEN order_date IS NULL THEN 1 END) as null_dates,
        COUNT(CASE WHEN order_time IS NULL THEN 1 END) as null_times
    FROM order_details
""")
result = cursor.fetchone()
print(f"\norder_details table:")
print(f"  Total records: {result[0]:,}")
print(f"  NULL item_ids: {result[1]} ({result[1]/result[0]*100:.2f}%)")
print(f"  NULL order_dates: {result[2]}")
print(f"  NULL order_times: {result[3]}")


MISSING VALUES ANALYSIS

order_details table:
  Total records: 12,234
  NULL item_ids: 137 (1.12%)
  NULL order_dates: 0
  NULL order_times: 0


In [25]:

# Missing values in menu_items
cursor.execute("""
    SELECT 
        COUNT(*) as total_items,
        COUNT(CASE WHEN item_name IS NULL THEN 1 END) as null_names,
        COUNT(CASE WHEN category IS NULL THEN 1 END) as null_categories,
        COUNT(CASE WHEN price IS NULL THEN 1 END) as null_prices
    FROM menu_items
""")
result = cursor.fetchone()
print(f"\nmenu_items table:")
print(f"  Total items: {result[0]}")
print(f"  NULL item_names: {result[1]}")
print(f"  NULL categories: {result[2]}")
print(f"  NULL prices: {result[3]}")


menu_items table:
  Total items: 32
  NULL item_names: 0
  NULL categories: 0
  NULL prices: 0


Initial data profiling revealed that approximately 1.12% of order records contain missing item_id values. Because these records cannot be reliably linked to menu items, they are excluded from item-level revenue and performance analysis. The raw records are retained in the source table for completeness and auditability, while downstream analysis is conducted on a cleaned analytical view.

In [26]:
# Check for invalid data - prices, duplicates, unmatched items
print("\n" + "=" * 60)
print("DATA QUALITY CHECKS")
print("=" * 60)

# Price validation
cursor.execute("SELECT COUNT(*), MIN(price), MAX(price) FROM menu_items")
count, min_price, max_price = cursor.fetchone()
print(f"\nPrice range (menu_items): ${min_price:.2f} - ${max_price:.2f}")



DATA QUALITY CHECKS

Price range (menu_items): $5.00 - $19.95


In [27]:

# Check for items in orders that don't exist in menu_items
cursor.execute("""
    SELECT COUNT(*) as unmatched_items
    FROM order_details od
    WHERE od.item_id NOT IN (SELECT menu_item_id FROM menu_items)
    AND od.item_id IS NOT NULL
""")
unmatched = cursor.fetchone()[0]
print(f"Orders referencing non-existent items: {unmatched}")


Orders referencing non-existent items: 0


In [28]:

# Duplicate check
cursor.execute("SELECT COUNT(*), COUNT(DISTINCT order_details_id) FROM order_details")
total, distinct = cursor.fetchone()
print(f"Duplicate order_details_ids: {total - distinct}")

cursor.execute("SELECT COUNT(*), COUNT(DISTINCT menu_item_id) FROM menu_items")
total, distinct = cursor.fetchone()
print(f"Duplicate menu_item_ids: {total - distinct}")


Duplicate order_details_ids: 0
Duplicate menu_item_ids: 0


In [29]:

# Sample of records with NULL item_ids
print("\nSample records with NULL item_ids:")
cursor.execute("SELECT order_details_id, order_id, order_date FROM order_details WHERE item_id IS NULL LIMIT 5")
for row in cursor.fetchall():
    print(f"  - order_details_id: {row[0]}, order_id: {row[1]}, date: {row[2]}")


Sample records with NULL item_ids:
  - order_details_id: 122, order_id: 50, date: 1/1/23
  - order_details_id: 298, order_id: 125, date: 1/2/23
  - order_details_id: 358, order_id: 147, date: 1/3/23
  - order_details_id: 387, order_id: 161, date: 1/3/23
  - order_details_id: 470, order_id: 200, date: 1/3/23


In [30]:
# Summary statistics before cleaning
print("\n" + "=" * 60)
print("SUMMARY STATISTICS")
print("=" * 60)

# Unique orders
cursor.execute("SELECT COUNT(DISTINCT order_id) FROM order_details")
unique_orders = cursor.fetchone()[0]
print(f"\nUnique orders: {unique_orders:,}")


# Items per order
cursor.execute("""
    SELECT 
        AVG(items_per_order) as avg_items,
        MIN(items_per_order) as min_items,
        MAX(items_per_order) as max_items
    FROM (
        SELECT COUNT(*) as items_per_order
        FROM order_details
        WHERE item_id IS NOT NULL
        GROUP BY order_id
    )
""")
avg_items, min_items, max_items = cursor.fetchone()
print(f"Items per order: avg={avg_items:.2f}, min={min_items}, max={max_items}")


# Menu categories
cursor.execute("SELECT COUNT(DISTINCT category) FROM menu_items")
num_categories = cursor.fetchone()[0]
print(f"Menu categories: {num_categories}")

cursor.execute("""
    SELECT category, COUNT(*) as item_count
    FROM menu_items
    GROUP BY category
    ORDER BY item_count DESC
""")
print("\nItems by category:")
for row in cursor.fetchall():
    print(f"  - {row[0]}: {row[1]} items")


SUMMARY STATISTICS

Unique orders: 5,370
Items per order: avg=2.26, min=1, max=14
Menu categories: 4

Items by category:
  - Mexican: 9 items
  - Italian: 9 items
  - Asian: 8 items
  - American: 6 items


- Rather than modifying or deleting raw records, data quality issues were addressed by creating a cleaned analytical view. Records with missing item references were excluded from downstream analysis, while all source tables were retained unchanged to preserve data integrity and auditability.

In [35]:
# Check order_analysis table
print("=" * 60)
print("ORDER ANALYSIS TABLE")
print("=" * 60)

# Count records
cursor.execute("SELECT COUNT(*) FROM order_analysis")
count = cursor.fetchone()[0]
print(f"\nTotal records in order_analysis: {count:,}")

# Display first 10 records
print("\nFirst 10 records:")
cursor.execute("SELECT * FROM order_analysis LIMIT 10")
columns = [description[0] for description in cursor.description]
print(f"\nColumns: {', '.join(columns)}\n")

for row in cursor.fetchall():
    print(row)

# Check for NULL item_ids
cursor.execute("SELECT COUNT(*) FROM order_analysis WHERE item_id IS NULL")
null_count = cursor.fetchone()[0]
print(f"\nNULL item_ids: {null_count}")

# Verify item names are populated
cursor.execute("SELECT COUNT(*) FROM order_analysis WHERE item_name IS NOT NULL")
named_items = cursor.fetchone()[0]
print(f"Records with item names: {named_items:,}")

# Verify revenue is calculated
cursor.execute("SELECT COUNT(*) FROM order_analysis WHERE revenue IS NOT NULL")
revenue_count = cursor.fetchone()[0]
print(f"Records with revenue calculated: {revenue_count:,}")

ORDER ANALYSIS TABLE

Total records in order_analysis: 12,097

First 10 records:

Columns: order_id, order_date, order_time, item_id, item_name, category, price, quantity, revenue

(1, None, None, 109.0, 'Korean Beef Bowl', 'Asian', 17.95, 1, 17.95)
(2, None, None, 108.0, 'Tofu Pad Thai', 'Asian', 14.5, 1, 14.5)
(2, None, None, 124.0, 'Spaghetti', 'Italian', 14.5, 1, 14.5)
(2, None, None, 117.0, 'Chicken Burrito', 'Mexican', 12.95, 1, 12.95)
(2, None, None, 129.0, 'Mushroom Ravioli', 'Italian', 15.5, 1, 15.5)
(2, None, None, 106.0, 'French Fries', 'American', 7.0, 1, 7.0)
(3, None, None, 117.0, 'Chicken Burrito', 'Mexican', 12.95, 1, 12.95)
(3, None, None, 119.0, 'Chicken Torta', 'Mexican', 11.95, 1, 11.95)
(4, None, None, 117.0, 'Chicken Burrito', 'Mexican', 12.95, 1, 12.95)
(5, None, None, 117.0, 'Chicken Burrito', 'Mexican', 12.95, 1, 12.95)

NULL item_ids: 0
Records with item names: 12,097
Records with revenue calculated: 12,097


- All subsequent analysis is conducted at the order–item level, where each row represents a single menu item purchased within an order.

### Menu Performance

In [40]:
# Revenue & Volume by Item
cursor.execute("""
SELECT
    item_name,
    category,
    COUNT(*) AS times_ordered,
    SUM(revenue) AS total_revenue
FROM order_analysis
GROUP BY item_name, category
ORDER BY total_revenue DESC;
               """)

Revenue_by_item = cursor.fetchall()[0]
print(f"Revenue and Volume by Item: {Revenue_by_item}")  

Revenue and Volume by Item: ('Korean Beef Bowl', 'Asian', 588, 10554.600000000037)


In [ ]:
# Top 15 Revenue Generators
print("\n" + "=" * 80)
print("TOP 15 REVENUE-GENERATING ITEMS")
print("=" * 80)

cursor.execute("""
SELECT
    item_name,
    category,
    COUNT(*) AS times_ordered,
    ROUND(SUM(revenue), 2) AS total_revenue,
    ROUND(SUM(revenue) / COUNT(*), 2) AS avg_price_per_order
FROM order_analysis
GROUP BY item_name, category
ORDER BY total_revenue DESC
LIMIT 15
""")

print(f"{'Rank':<6} {'Item Name':<30} {'Category':<12} {'Orders':<10} {'Total Revenue':<18} {'Avg Price':<12}")
print("-" * 90)

for rank, row in enumerate(cursor.fetchall(), 1):
    item_name, category, times_ordered, total_revenue, avg_price = row
    print(f"{rank:<6} {item_name:<30} {category:<12} {times_ordered:<10} ${total_revenue:>15,.2f}  ${avg_price:>10,.2f}")


TOP 15 REVENUE-GENERATING ITEMS
Rank   Item Name                      Category     Orders     Total Revenue      Avg Price   
------------------------------------------------------------------------------------------
1      Korean Beef Bowl               Asian        588        $      10,554.60  $     17.95
2      Spaghetti & Meatballs          Italian      470        $       8,436.50  $     17.95
3      Tofu Pad Thai                  Asian        562        $       8,149.00  $     14.50
4      Cheeseburger                   American     583        $       8,132.85  $     13.95
5      Hamburger                      American     622        $       8,054.90  $     12.95
6      Orange Chicken                 Asian        456        $       7,524.00  $     16.50
7      Eggplant Parmesan              Italian      420        $       7,119.00  $     16.95
8      Steak Torta                    Mexican      489        $       6,821.55  $     13.95
9      Chicken Parmesan               Italian 

In [43]:
# Bottom 10 Worst Performers
print("\n" + "=" * 80)
print("BOTTOM 10 LOWEST-REVENUE ITEMS (CANDIDATES FOR REMOVAL)")
print("=" * 80)

cursor.execute("""
SELECT
    item_name,
    category,
    COUNT(*) AS times_ordered,
    ROUND(SUM(revenue), 2) AS total_revenue,
    ROUND(SUM(revenue) / COUNT(*), 2) AS avg_price_per_order
FROM order_analysis
GROUP BY item_name, category
ORDER BY total_revenue ASC
LIMIT 10
               """)
Worst_performers = cursor.fetchall()
print(f"{'Rank':<6} {'Item Name':<30} {'Category':<12} {'Orders':<10} {'Total Revenue':<18} {'Avg Price':<12}")
for rank, row in enumerate(Worst_performers, 1):
    item_name, category, times_ordered, total_revenue, avg_price = row
    print(f"{rank:<6} {item_name:<30} {category:<12} {times_ordered:<10} ${total_revenue:>15,.2f}  ${avg_price:>10,.2f}")




BOTTOM 10 LOWEST-REVENUE ITEMS (CANDIDATES FOR REMOVAL)
Rank   Item Name                      Category     Orders     Total Revenue      Avg Price   
1      Chicken Tacos                  Mexican      123        $       1,469.85  $     11.95
2      Potstickers                    Asian        205        $       1,845.00  $      9.00
3      Chips & Guacamole              Mexican      237        $       2,133.00  $      9.00
4      Hot Dog                        American     257        $       2,313.00  $      9.00
5      Cheese Quesadillas             Mexican      233        $       2,446.50  $     10.50
6      Veggie Burger                  American     238        $       2,499.00  $     10.50
7      Steak Tacos                    Mexican      214        $       2,985.30  $     13.95
8      Edamame                        Asian        620        $       3,100.00  $      5.00
9      Cheese Lasagna                 Italian      207        $       3,208.50  $     15.50
10     Chips & Salsa 

In [ ]:
#Top 15 Revenue Generators
pd.read_sql_query("""
SELECT
    item_name,
    category,
    COUNT(*) AS times_ordered,
    ROUND(SUM(revenue), 2) AS total_revenue,
    ROUND(SUM(revenue) / COUNT(*), 2) AS avg_price_per_order
FROM order_analysis
GROUP BY item_name, category
ORDER BY total_revenue DESC
LIMIT 15
""", conn)


,item_name,category,times_ordered,total_revenue,avg_price_per_order
0,Korean Beef Bowl,Asian,588,10554.60,17.95
1,Spaghetti & Meatballs,Italian,470,8436.50,17.95
2,Tofu Pad Thai,Asian,562,8149.00,14.50
3,Cheeseburger,American,583,8132.85,13.95
4,Hamburger,American,622,8054.90,12.95
5,Orange Chicken,Asian,456,7524.00,16.50
6,Eggplant Parmesan,Italian,420,7119.00,16.95
7,Steak Torta,Mexican,489,6821.55,13.95
8,Chicken Parmesan,Italian,364,6533.80,17.95
9,Pork Ramen,Asian,360,6462.00,17.95
